In [ ]:
#Copyright 2025 Er. Aditya Nath Thakur.
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [78]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [47]:
#⚙️ Section 1: Setup
#1.1: Install dependencies
#The Kaggle Notebooks environment includes a pre-installed version of the google-adk library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

#To install and use ADK in your own Python development environment outside of this course, you can do so by running:***

In [157]:
pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [50]:
#2. Add the key to Kaggle Secrets

#Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

#In the top menu bar of the notebook editor, select Add-ons then Secrets.
#Create a new secret with the label GOOGLE_API_KEY.
#Paste your API key into the "Value" field and click "Save".
#Ensure that the checkbox next to GOOGLE_API_KEY is selected so that the secret is attached to the notebook.
#3. Authenticate in the notebook

#Run the cell below to complete authentication.

In [158]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [51]:
#1.3: Import ADK components
#Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [159]:
import uuid
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, FunctionTool, ToolContext
from google.genai import types
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [52]:
#1.4: Helper functions
#We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [160]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [54]:
#1.5: Configure Retry Options
#When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.***

In [161]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [162]:
# Install the filesystem server globally so it starts instantly
!npm install -g @modelcontextprotocol/server-filesystem
print("✅ MCP Filesystem Server installed.")

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
changed 129 packages in 4s
⠼
⠼35 packages are looking for funding
⠼  run `npm fund` for details
⠼✅ MCP Filesystem Server installed.


In [163]:
# Define the directory where Kaggle allows writing
kaggle_output_dir = "/kaggle/working"

mcp_filesystem_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx", 
            args=[
                "-y",
                "@modelcontextprotocol/server-filesystem", 
                kaggle_output_dir, # <--- MANDATORY SANDBOX ARGUMENT
            ],
            # We only allow writing and listing
            tool_filter=["write_file", "list_directory"], 
        ),
        # Increased timeout to allow Node.js to startup
        timeout=120, 
    )
)

print("✅ MCP Toolset configured for Kaggle.")

✅ MCP Toolset configured for Kaggle.


In [164]:
import os
import sys
from kaggle_secrets import UserSecretsClient
from google.adk.models import Gemini
import google.auth

# 1. Retrieve the JSON key from Kaggle Secrets
try:
    user_secrets = UserSecretsClient()
    # Make sure the label matches what you typed in Step 2 ('GCP_JSON_KEY')
    key_content = user_secrets.get_secret("GCP_JSON_KEY") 
    
    # 2. Write it to a temporary file
    json_path = "/tmp/gcp_key.json"
    with open(json_path, "w") as f:
        f.write(key_content)
        
    # 3. Set the environment variable so Google Auth finds it
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path
    
    # 4. Authenticate
    credentials, project_id = google.auth.default()
    print(f"✅ Kaggle Authenticated with Project: {project_id}")

except Exception as e:
    print("❌ Authentication Failed. Did you add the 'GCP_JSON_KEY' to Kaggle Secrets?")
    print(e)

# --- NOW DEFINE YOUR MODEL ---

# Hardcode your location since the key file usually doesn't contain it
location = "us-central1" 

shared_gemini_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=None,
    vertexai=True,
    project=project_id, 
    location=location
)

print("✅ Shared Model Ready")

✅ Kaggle Authenticated with Project: gen-lang-client-0658909834
✅ Shared Model Ready


In [165]:
import google.auth
from google.adk.models import Gemini

# 1. Automatically get your Google Cloud Project ID
credentials, project_id = google.auth.default()

# 2. Set your region (Change 'us-central1' if you are in a different region like 'asia-south1')
location = "asia-south1" 

print(f"✅ Authenticating with Project: {project_id} in Location: {location}")

✅ Authenticating with Project: gen-lang-client-0658909834 in Location: asia-south1


In [ ]:
#These are the main properties we'll set:

#name and description: A simple name and description to identify our agent.
#model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
#instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
#tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.

In [166]:
# --- STEP 1: CREATE THE SHARED MODEL FIRST ---
retry_config = None

# We define the variable 'shared_model' here
shared_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
    vertexai=True,
    project="gen-lang-client-0658909834",  # Your Project ID
    location="asia-south1"                 # Your Region
)

print("✅ Shared Model defined successfully.")

✅ Shared Model defined successfully.


In [167]:
# Research Agent: Its Job is to use the google_search tool and present findings 
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
        description="Use Google Search to find relevant Indian Road Congress (IRC) Codes, IRC Special Publication (IRC:SP) Codes, and Indian Standard (IS) Codes that govern the user's specific query topic. Also Find Global Research & Innovation",
        instruction="""For every user query related to a bridge , building or road construction and maintenance topic in India (e.g., "design of flexible pavement," "foundation for a major bridge," "use of high-performance concrete," etc.), follow this mandatory, sequential workflow:

Step 1: Identify and Locate Indian Codes (The Core Task)
Search Protocol: Use Google Search to find relevant Indian Road Congress (IRC) Codes, IRC Special Publication (IRC:SP) Codes, and Indian Standard (IS) Codes that govern the user's specific query topic.

Focus: Prioritize finding the most recent revision or amendment of the relevant codes.

Step 2: Extract Code-Specific Directives (The Clause Detail)
For the top 3-5 most relevant codes identified in Step 1, perform a targeted search for the internal content.

Extraction Requirement: For each relevant code, you MUST identify the following information and present it in a table format (as described in Step 4):

Code Number and Year (e.g., IRC:37-2018).

The Clause Number or Section/Paragraph most directly related to the user's question (e.g., Clause 4.3.2).

A Concise Summary (1-2 sentences) of the exact directive, rule, or specification provided in that specific clause/section.

Step 3: Determine Recent Updates and Revisions (The Compliance Check)
Search specifically for any amendments, circulars, or recent revisions (within the last 5 years) related to the codes identified in Step 1.

Mandatory Inclusion: Explicitly state whether the code is the latest version, or if there have been any significant updates or amendments. If an update exists, state the year of the amendment and the nature of the change (e.g., increase in design load, new material specification, revised safety factor).

Step 4: Find Global Research & Innovation (The Forward Look)
Broaden the search to the latest research and findings (last 3 years) from prominent international organizations and technical journals on the user's topic (e.g., AASHTO, Eurocodes, research on sustainable materials, digital construction, AI in infrastructure).

Synthesize: Provide a brief, high-level explanation of this global research and how it relates to or could potentially influence future Indian codes.

Step 5: Format and Final Presentation (The Output Structure)
Structure the final response using clear Markdown headings and tables for maximum readability and citable detail.

Final Output Format:
Concise Answer: Start with a brief, direct summary of the key takeaway for the user's question.

Code Compliance: Indian Standards: (Use a table for details from Step 2).

Code Revision Status: (Details from Step 3).

Global Research & Future Trends: (Synthesized details from Step 4).

3. Formatting Guidelines
Style: Professional, technical, precise, and highly informative.

Emphasize: Use bolding to highlight code numbers, clause numbers, and key technical terms.

Table Requirement: The table in the "Code Compliance: Indian Standards" section is MANDATORY. Use the following column headers: | Code No. & Year | Relevant Clause No. | Directive/Specification Summary | | :--- | :--- | :--- |
""", output_key="agent_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Research Agent defined.")


✅ Research Agent defined.


In [168]:
#Agent 2: Compliance Validator
#This agent receives the raw data from the Research Agent (Agent 1) and validates its technical content.
Compliance_Validator_Agent = Agent(
    name="ComplianceValidatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
            instruction=""" Following this outline strictly: {agent_research}
            The Research Agent (Agent 1) has provided a list of code directives, clauses, and summaries. 
            Follow this mandatory, sequential workflow to ensure technical accuracy:

            Step 1: Conflict and Hierarchy Check
            Review all extracted directives (e.g., from IRC, IS, and global codes). If multiple clauses address the same design parameter (e.g., span-to-depth ratio or safety factor), identify and output the **most conservative (safest/most stringent) rule**. State the conflict and the rationale for the selection.

            Step 2: Numerical Verification
            Focus on all numerical values (e.g., concrete grade M30, wind speed 47 m/s, minimum steel ratio 0.8%). Use an internal knowledge base or perform a targeted search (if necessary) to verify the general acceptance and correct units of the extracted numerical data.

            Step 3: Output Validation Summary
            Generate a concise, technical report detailing any identified conflicts, the rule chosen for final recommendation, and a confirmation of data integrity.
            """,
    output_key="validator_compliance",
)

print("✅ Compliance_Validator_Agent created.")

✅ Compliance_Validator_Agent created.


In [169]:
# Agent 3: Technical Refiner
#This agent focuses purely on the presentation and language quality of the final response, ensuring it meets professional standards.
Technical_Refiner_Agent = Agent(
    name="TechnicalRefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction=""" Following this outline strictly: {validator_compliance}
            The previous agents have completed data retrieval and technical validation. 
            Follow this mandatory, sequential workflow to finalize the output:

            Step 1: Synthesize and Contextualize
            Combine the key findings from the Code Compliance Table (Agent 1) and the Validation Report (Agent 2). Convert the tabular data and key validation points into **fluent, citable advisory paragraphs** under the relevant Markdown headings.

            Step 2: Ensure Professional Tone
            Ensure the language is **professional, precise, and objective**. Use appropriate technical civil engineering terminology. Remove any conversational language or internal workflow notes.

            Step 3: Final Formatting Check
            Strictly adhere to all formatting requirements of the Root Agent (e.g., mandatory table, bolding of code numbers, clear headings). Ensure the final response is ready for direct user presentation.
            """,
    output_key="final_summary",
)

print("✅ Technical_Refiner_Agent created.")

✅ Technical_Refiner_Agent created.


In [170]:
# Ensure shared_model is defined from previous steps
pdf_agent = LlmAgent(
    name="pdf_agent",
    model=shared_model,
    instruction="""
    **Role:** Document Persistence Agent.
    
    **Objective:** Save the final report provided by the previous agent to the disk using the MCP Tool.
    
    **Strict Tool Usage Rules:**
    1. You MUST use the tool `write_file`.
    2. **Path:** You MUST use the absolute path: `/kaggle/working/Final_Report.md`.
       (DO NOT use './' or just the filename. The tool will fail.)
    3. **Content:** Paste the full text report you received.
    
    **Confirmation:**
    After calling the tool, confirm to the user: "Report saved to /kaggle/working/Final_Report.md".
    """,
    tools=[mcp_filesystem_server],
)

print("✅ PDF Agent (MCP) defined.")

✅ PDF Agent (MCP) defined.


In [171]:
root_agent = SequentialAgent(
    name="CivilEngineeringPipeline",
  
    sub_agents=[research_agent, Compliance_Validator_Agent, Technical_Refiner_Agent, pdf_agent ],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [173]:
import os

# List all files in the Kaggle output directory
output_dir = "/kaggle/working"
files = os.listdir(output_dir)

print(f"📂 Files found in {output_dir}:")
for f in files:
    print(f" - {f}")

# Look specifically for markdown or text files
target_file = None
for f in files:
    if f.endswith(".md") or f.endswith(".txt"):
        target_file = f
        break

if target_file:
    print(f"\n✅ SUCCESS: Found target file: {target_file}")
else:
    print("\n❌ No Report found. The agent might not have saved it yet.")

📂 Files found in /kaggle/working:
 - root_agent
 - .virtual_documents

❌ No Report found. The agent might not have saved it yet.


In [ ]:
#2.3 Run your agent
#Now it's time to bring your agent to life and send it a query. To do this, you need a Runner, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

#a. Create an InMemoryRunner and tell it to use our root_agent:

In [174]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [55]:
#b. Now you can call the .run_debug() method to send our prompt and get an answer.

#👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [175]:
response = await runner.run_debug(
    "details about how to construct hill roads"
)


 ### Created new session: debug_session_id

User > details about how to construct hill roads
ResearchAgent > ResearchAgent's response:

**Concise Answer:**
Constructing hill roads in India requires adherence to specific guidelines that address the unique challenges posed by varying terrain, geological conditions, and environmental factors. The primary Indian code governing the design and construction of hill roads is the **IRC:SP:48-2011**, "Hill Roads Manual." This manual provides comprehensive directives on planning, alignment, geometric design, construction methods, slope stabilization, drainage, and maintenance of roads in hilly and mountainous regions.

**Code Compliance: Indian Standards**

| Code No. & Year | Relevant Clause No. | Directive/Specification Summary |
| :-------------- | :------------------ | :------------------------------ |
| **IRC:SP:48-2011** | Section 1: Introduction | Defines hill roads and their classification, emphasizing the need for specific design consid

/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79d7d9917ad0>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79d7d9c6ac50>
ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x79d7d98fb2d0>


pdf_agent > I have saved the report to `/kaggle/working/Final_Report.md`.


In [178]:
import os
from IPython.display import FileLink

# 1. Force the notebook to look in the correct directory
os.chdir("/kaggle/working")

# 2. Define the filename (Relative path, NO slashes)
filename = "Final_Report.md"

# 3. Check and Link
if os.path.exists(filename):
    print(f"✅ File found: {filename}")
    print(f"📏 Size: {os.path.getsize(filename)} bytes")
    
    # Pass ONLY the filename, not the full path
    display(FileLink(filename))
else:
    print("❌ File not found. The MCP server did not write it.")
    # Debug: Show what IS there
    print("Current files:", os.listdir("."))

✅ File found: Final_Report.md
📏 Size: 4187 bytes


/kaggle/working/Final_Report.md

In [ ]:
#🚀 2.5 Your Turn!
#This is your chance to see the agent in action. Ask it a question that requires current information.

#Try one of these, or make up your own:
# details about the bridge maintenance 
# details of bridge construction 
#etc

In [99]:
!adk create root_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/root_agent:
- .env
- __init__.py
- agent.py



In [100]:
url_prefix = get_adk_proxy_url()

In [101]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [183]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: